In [81]:
import numpy as np
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize

In [82]:
# from newspaper import Article
# # get article
# url1 = "http://www.nytimes.com/2016/05/05/technology/moores-law-running-out-of-room-tech-looks-for-a-successor.html?smid=pl-share"
# article1 = Article(url1)
# article1.download()
# article1.parse()
# raw1 = article1.text.lower()
# raw1 = raw1.encode("utf-8")
# raw1.replace(".", "")
# raw1.replace(",", "")

In [128]:
with open('obama.txt', 'r') as myfile:
    data1 = myfile.read().replace('\n', '')
data1 = data1.lower()
data1 = data1.replace(".", "")
data1 = data1.replace(",", "")
tokens1 = word_tokenize(data1)
sents1 = sent_tokenize(data1)
text_len1 = len(tokens1)
fdist1 = FreqDist(tokens1)

In [136]:
# make cooccurence array
array1 = np.zeros((text_len1, text_len1))

In [109]:
# populate coocurrence array
fdist1.items()[:5] #5 most frequent items in fdist

#to do: figure out how to do coocurrence.. 

[('the', 124), ('and', 108), ('of', 82), ('to', 66), ('our', 57)]

In [ ]:
# looking through each document 
def examineSentence(sent):
    # for word1 in sent:
    #     for words 2 in sent:
    #         how frequently does word2 cooccur with word1 (look at coocurrence array)
    #         return that value
    return